## Project # 2

Create a Multiprocessing, Threaded and Asynchronous application that will calculate the highest prime number in 3 min.

You must start a 0

After you have created these programs analyze  the performance of each. IE: How long did it take to find prime number? Compare each prime and why each found that prime number. 

If you wish you can calculate any other non dependent item and them perform analyses.

For code consistancy use the following is_prime function.
```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True
```
    async       13,386,001
    threaded(2) 10,747,921
    

In [1]:
def getStuff (Lists):
    list1 = []
    for List in Lists:
        if (Lists[0] == 4):
            list1.append(List)
    return list1


In [2]:
getStuff([1,2,3,4,5])

[]

In [1]:
a = 1
for i in range(a,4):
    print(i)

    print(i)

1
1
2
2
3
3


# Huynh Gia Phong Tat (Jayden)


## Project 2

### Multiprocessing

#### Project 2 Multiprocessing

In [ ]:
import time
from multiprocessing import Process, Value, Lock
import math
import sys

# Python 3.11+ won't let you convert huge integers to strings unless you raise the limit.
# I ran into this trying to print Fib(1000000), so thanks to Stack Overflow for the fix.
sys.set_int_max_str_digits(1000000)

# Basic prime checker using trial division.
# It's not fast for huge numbers, but it works — and I needed something reliable.
def _is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Each process runs this function to search for primes.
# I split the search space across multiple processes using different start points and steps.
# If a larger prime is found, I update the shared variable with a lock to avoid conflicts.
def search_primes(start, step, largest_prime, lock, end_time):
    n = start
    while time.time() < end_time:
        if _is_prime(n):
            with lock:
                if n > largest_prime.value:
                    largest_prime.value = n
        n += step

# Iterative Fibonacci function.
# I capped it because trying to compute something like Fib(50000000) made my machine beg for mercy.
def compute_fibonacci(n):
    MAX_FIB = 1000000
    if n > MAX_FIB:
        print(f"[Fibonacci] Prime too large, capping n from {n} to {MAX_FIB}")
        n = MAX_FIB

    print(f"[Fibonacci] Calculating Fib({n})...")
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
        if i % 200000 == 0:
            print(f"  Reached Fib({i})")
    print(f"[Fibonacci] Done. Fib({n}) has {len(str(a))} digits.")

# Factorial computation.
# I initially didn’t expect how massive factorials get — thank you Stack Overflow again.
def compute_factorial(n):
    MAX_FACT = 100000
    if n > MAX_FACT:
        print(f"[Factorial] Prime too large, capping n from {n} to {MAX_FACT}")
        n = MAX_FACT

    print(f"[Factorial] Calculating {n}!...")
    result = math.factorial(n)
    print(f"[Factorial] Done. {n}! has {len(str(result))} digits.")

# Main function to control the flow.
# I used multiple processes to search for primes in parallel over 3 minutes.
# Once a prime is found, I run Fibonacci and factorial calculations at the same time.
def main():
    NUM_PROCESSES = 7
    SEARCH_DURATION = 3 * 60  # 3 minutes

    largest_prime = Value('i', 0)
    lock = Lock()
    end_time = time.time() + SEARCH_DURATION

    processes = []
    for i in range(NUM_PROCESSES):
        p = Process(target=search_primes, args=(i, NUM_PROCESSES, largest_prime, lock, end_time))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    prime_found = largest_prime.value
    print(f"\nLargest prime found in 3 minutes: {prime_found}\n")

    # Run the two heavy computations in parallel (learned this trick from multiprocessing docs and forums)
    fib_proc = Process(target=compute_fibonacci, args=(prime_found,))
    fact_proc = Process(target=compute_factorial, args=(prime_found,))

    fib_proc.start()
    fact_proc.start()

    fib_proc.join()
    fact_proc.join()

    print("\nAll tasks completed.")

if __name__ == "__main__":
    start = time.time()
    main()
    print(f"\nTotal runtime: {time.time() - start:.2f} seconds")


Largest prime found in 3 minutes: 60006767
<br>
[Factorial] Prime too large, capping n from 60006767 to 100000
<br>
[Factorial] Calculating 100000!...
<br>
[Fibonacci] Prime too large, capping n from 60006767 to 1000000
<br>
[Fibonacci] Calculating Fib(1000000)...
<br>
  Reached Fib(0)
  <br>
  Reached Fib(200000)
  <br>
[Factorial] Done. 100000! has 456574 digits.
<br>
  Reached Fib(400000)
  <br>
  Reached Fib(600000)
  <br>
  Reached Fib(800000)
<br>
[Fibonacci] Done. Fib(1000000) has 208988 digits.
<br>
All tasks completed.
<br>
Total runtime: 186.13 seconds

### Threading

#### Project 2 Threading

In [6]:
import time
import threading
import math
import sys

# Same Python 3.11 issue as before — need to increase the digit limit for giant integers
sys.set_int_max_str_digits(1000000)

# Basic prime check using trial division
# Again, not optimized for speed, but reliable enough for this kind of use
def _is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Global variables shared between threads
largest_prime = 0
lock = threading.Lock()

# Each thread runs this function to check numbers in a slice of the range
# I use a lock to make sure no two threads try to update largest_prime at the same time
def search_primes(start, step, end_time):
    global largest_prime
    n = start
    while time.time() < end_time:
        if _is_prime(n):
            with lock:
                if n > largest_prime:
                    largest_prime = n
        n += step

# Iterative Fibonacci — capped to avoid memory explosion
def compute_fibonacci(n):
    MAX_FIB = 1000000
    if n > MAX_FIB:
        print(f"[Fibonacci] Prime too large, capping n from {n} to {MAX_FIB}")
        n = MAX_FIB

    print(f"[Fibonacci] Calculating Fib({n})...")
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
        if i % 200000 == 0:
            print(f"  Reached Fib({i})")
    print(f"[Fibonacci] Done. Fib({n}) has {len(str(a))} digits.")

# Built-in factorial with a cap — thanks again Stack Overflow for helping me understand how fast this grows
def compute_factorial(n):
    MAX_FACT = 100000
    if n > MAX_FACT:
        print(f"[Factorial] Prime too large, capping n from {n} to {MAX_FACT}")
        n = MAX_FACT

    print(f"[Factorial] Calculating {n}!...")
    result = math.factorial(n)
    print(f"[Factorial] Done. {n}! has {len(str(result))} digits.")

# Main threading logic
def main():
    NUM_THREADS = 10
    SEARCH_DURATION = 3 * 60  # 3 minutes

    end_time = time.time() + SEARCH_DURATION
    threads = []

    # Start each thread with a unique offset and shared step to divide the range
    for i in range(NUM_THREADS):
        t = threading.Thread(target=search_primes, args=(i, NUM_THREADS, end_time))
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    prime_found = largest_prime
    print(f"\nLargest prime found in 3 minutes: {prime_found}\n")

    # Start Fibonacci and Factorial threads after the prime is found
    fib_thread = threading.Thread(target=compute_fibonacci, args=(prime_found,))
    fact_thread = threading.Thread(target=compute_factorial, args=(prime_found,))

    fib_thread.start()
    fact_thread.start()

    fib_thread.join()
    fact_thread.join()

    print("\nAll tasks completed.")

if __name__ == "__main__":
    start = time.time()
    main()
    print(f"\nTotal runtime: {time.time() - start:.2f} seconds")



Largest prime found in 3 minutes: 10663867

[Fibonacci] Prime too large, capping n from 10663867 to 1000000
[Fibonacci] Calculating Fib(1000000)...
  Reached Fib(0)
[Factorial] Prime too large, capping n from 10663867 to 100000
[Factorial] Calculating 100000!...
  Reached Fib(200000)
[Factorial] Done. 100000! has 456574 digits.
  Reached Fib(400000)
  Reached Fib(600000)
  Reached Fib(800000)
[Fibonacci] Done. Fib(1000000) has 208988 digits.

All tasks completed.

Total runtime: 187.06 seconds


### Asychronous

#### Project 2 Asynchronous

In [ ]:
import time
import asyncio
import math
import sys

# Once again, Python 3.11+ prevents you from converting huge integers to strings by default
# Found the fix on Stack Overflow (bless that place)
sys.set_int_max_str_digits(1000000)

# Trial division for prime check. Not fast, but reliable.
def _is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# Shared value for the largest prime
# Using a regular int here since asyncio doesn't do true parallelism anyway
largest_prime = 0
lock = asyncio.Lock()

# Async wrapper for checking primes
# Not actually parallel (thanks to the GIL), but this is how you do async workloads
async def check_primes(start, step, end_time):
    global largest_prime
    n = start
    while time.time() < end_time:
        if _is_prime(n):
            async with lock:
                if n > largest_prime:
                    largest_prime = n
        n += step
        await asyncio.sleep(0)  # Yield control so other tasks can run

# Same Fibonacci function as before, capped to prevent meltdown
async def compute_fibonacci(n):
    MAX_FIB = 1000000
    if n > MAX_FIB:
        print(f"[Fibonacci] Prime too large, capping n from {n} to {MAX_FIB}")
        n = MAX_FIB

    print(f"[Fibonacci] Calculating Fib({n})...")
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
        if i % 200000 == 0:
            print(f"  Reached Fib({i})")
        await asyncio.sleep(0)  # Yield control
    print(f"[Fibonacci] Done. Fib({n}) has {len(str(a))} digits.")

# Factorial with the usual cap
async def compute_factorial(n):
    MAX_FACT = 100000
    if n > MAX_FACT:
        print(f"[Factorial] Prime too large, capping n from {n} to {MAX_FACT}")
        n = MAX_FACT

    print(f"[Factorial] Calculating {n}!...")
    result = math.factorial(n)
    print(f"[Factorial] Done. {n}! has {len(str(result))} digits.")

# Main async function
async def main():
    global largest_prime
    NUM_TASKS = 10
    SEARCH_DURATION = 3 * 60
    end_time = time.time() + SEARCH_DURATION

    # Kick off prime-checking tasks
    prime_tasks = []
    for i in range(NUM_TASKS):
        prime_tasks.append(asyncio.create_task(check_primes(i, NUM_TASKS, end_time)))

    await asyncio.gather(*prime_tasks)

    print(f"\nLargest prime found in 3 minutes: {largest_prime}\n")

    # Run Fibonacci and Factorial concurrently once the prime is known
    fib_task = asyncio.create_task(compute_fibonacci(largest_prime))
    fact_task = asyncio.create_task(compute_factorial(largest_prime))

    await asyncio.gather(fib_task, fact_task)

    print("\nAll tasks completed.")

#Took this from chatgpt, it allows the code to run in jupyter notebook without blocking the kernel
# and allows for multiple calls to asyncio.run() in the same notebook cell
# I also provide the non chatgpt version as script in this folder
if __name__ == "__main__":
    import nest_asyncio
    import asyncio

    nest_asyncio.apply()
    loop = asyncio.get_event_loop()

    start = time.time()
    task = main() 
    loop.run_until_complete(task)
    print(f"\nTotal runtime: {time.time() - start:.2f} seconds")




Largest prime found in 3 minutes: 16719541

[Fibonacci] Prime too large, capping n from 16719541 to 1000000
[Fibonacci] Calculating Fib(1000000)...
  Reached Fib(0)
[Factorial] Prime too large, capping n from 16719541 to 100000
[Factorial] Calculating 100000!...
[Factorial] Done. 100000! has 456574 digits.
  Reached Fib(200000)
  Reached Fib(400000)
  Reached Fib(600000)
  Reached Fib(800000)
[Fibonacci] Done. Fib(1000000) has 208988 digits.

All tasks completed.

Total runtime: 191.40 seconds


C:\Users\Phong\AppData\Local\Temp\ipykernel_29204\2412009117.py:101: RuntimeWarning: coroutine 'main' was never awaited
  loop.run_until_complete(main())


Output from the script
<br>
Largest prime found in 3 minutes: 17780339
<br>
[Fibonacci] Prime too large, capping n from 17780339 to 1000000
<br>
[Fibonacci] Calculating Fib(1000000)...
<br>
  Reached Fib(0)
  <br>
[Factorial] Prime too large, capping n from 17780339 to 100000
<br>
[Factorial] Calculating 100000!...
<br>
[Factorial] Done. 100000! has 456574 digits.
<br>
  Reached Fib(200000)
  <br>
  Reached Fib(400000)
  <br>
  Reached Fib(600000)
  <br>
  Reached Fib(800000)
<br>
[Fibonacci] Done. Fib(1000000) has 208988 digits.
<br>
All tasks completed.
<br>
Total runtime: 188.76 seconds